In [69]:
import pandas as pd
import numpy as np
import plotly.express as px

In [70]:
df = pd.read_excel('get_around_delay_analysis.xlsx')

In [71]:
df

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN
...,...,...,...,...,...,...,...
21305,573446,380069,mobile,ended,NaN,573429.0,300.0
21306,573790,341965,mobile,ended,-337.0,NaN,NaN
21307,573791,364890,mobile,ended,144.0,NaN,NaN
21308,574852,362531,connect,ended,-76.0,NaN,NaN


In [72]:
df.describe(include='all')

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,21310,21310,16346.000000,1841.000000,1841.000000
unique,NaN,NaN,2,2,NaN,NaN,NaN
top,NaN,NaN,mobile,ended,NaN,NaN,NaN
freq,NaN,NaN,17003,18045,NaN,NaN,NaN
mean,549712.880338,350030.603426,NaN,NaN,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,NaN,NaN,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,NaN,NaN,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,NaN,NaN,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,NaN,NaN,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,NaN,NaN,67.000000,560823.000000,540.000000


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  int64  
 1   car_id                                      21310 non-null  int64  
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.1+ MB


In [74]:
df = df.rename({'delay_at_checkout_in_minutes': 'delay_checkout','previous_ended_rental_id': 'previous_rental_id', 'time_delta_with_previous_rental_in_minutes': 'delta_previous'}, axis=1)

In [75]:
px.pie(df, names='state', title='Part of ended and cancelled rental')

In [76]:
px.pie(df, names='checkin_type', title='Part of mobile and connect checkin')

In [77]:
df['rental_id'].nunique()

21310

In [78]:
df['car_id'].nunique()

8143

In [79]:
df_rental_bycar = df.groupby('car_id').agg({'rental_id':'count'}).sort_values('rental_id',ascending=False)

In [80]:
df_rental_bycar = df_rental_bycar.rename({'rental_id':'rental_nb'}, axis=1)

In [81]:
df_rental_bycar

,rental_nb
car_id,
334190,33
241021,29
392479,29
342699,27
300505,24
...,...
324680,1
388501,1
388548,1


In [82]:
px.histogram(df_rental_bycar, x='rental_nb', title='Distribution of rental number for one car', nbins=50)

In [83]:
px.box(df_rental_bycar, x='rental_nb', title='Distribution of rental number for one car')

In [84]:
# calculation en % of outliers
outlier_limit = 7

nb_outlier = df_rental_bycar.query('rental_nb > @outlier_limit').shape[0]
total_cars = df_rental_bycar.shape[0]
print('% of outliers are', round((nb_outlier / total_cars) * 100,2))

% of outliers are 4.92


In [85]:
# todo : delete outliers ?

### Checkout analysis

In [86]:
px.box(df, x='delay_checkout', title='Distribution of checkout delay')

In [87]:
nb_outlier = df.query('(delay_checkout > 7000) | (delay_checkout < -2900)').shape[0]
total_rental = df.shape[0]
print('% of outliers for rental are', round((nb_outlier / total_rental) * 100, 2))

% of outliers for rental are 0.13


In [88]:
df

,rental_id,car_id,checkin_type,state,delay_checkout,previous_rental_id,delta_previous
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN
...,...,...,...,...,...,...,...
21305,573446,380069,mobile,ended,NaN,573429.0,300.0
21306,573790,341965,mobile,ended,-337.0,NaN,NaN
21307,573791,364890,mobile,ended,144.0,NaN,NaN
21308,574852,362531,connect,ended,-76.0,NaN,NaN


In [89]:
df_prev = df[~df['previous_rental_id'].isna()]

In [90]:
df_prev['delta_previous'].isna().sum()
# results at 0 mean tha when previous_ended_rental_id is set, then time_delta_with_previous_rental_in_minutes is set

0

In [91]:
# I want to know for a rental_id, the next rental_id if exist (based on previous_rental_id then)

In [92]:
df_prev['previous_rental_id'] = df_prev['previous_rental_id'].astype(int)

/var/folders/j3/n3dz9by90yd9388qpd2nsgxm0000gn/T/ipykernel_32578/4030823329.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [93]:
df_prev

,rental_id,car_id,checkin_type,state,delay_checkout,previous_rental_id,delta_previous
6,511639,370585,connect,ended,-15.0,563782,570.0
19,519491,312389,mobile,ended,58.0,545639,420.0
23,521156,392479,mobile,ended,NaN,537298,0.0
34,525044,349751,mobile,ended,NaN,510607,60.0
40,528808,181625,connect,ended,-76.0,557404,330.0
...,...,...,...,...,...,...,...
21269,568049,381499,connect,canceled,NaN,562174,720.0
21272,568241,396409,mobile,canceled,NaN,566136,570.0
21275,568523,297973,mobile,ended,12.0,567121,240.0
21286,569717,377312,mobile,ended,230.0,545045,90.0


In [94]:
df_prev['previous_rental_id'].nunique()

1788

In [95]:
# df_prev = df_prev.rename({'rental_id': 'next_rental_id', 'previous_rental_id':'rental_id'}, axis=1)

In [96]:
df_prev

,rental_id,car_id,checkin_type,state,delay_checkout,previous_rental_id,delta_previous
6,511639,370585,connect,ended,-15.0,563782,570.0
19,519491,312389,mobile,ended,58.0,545639,420.0
23,521156,392479,mobile,ended,NaN,537298,0.0
34,525044,349751,mobile,ended,NaN,510607,60.0
40,528808,181625,connect,ended,-76.0,557404,330.0
...,...,...,...,...,...,...,...
21269,568049,381499,connect,canceled,NaN,562174,720.0
21272,568241,396409,mobile,canceled,NaN,566136,570.0
21275,568523,297973,mobile,ended,12.0,567121,240.0
21286,569717,377312,mobile,ended,230.0,545045,90.0


In [97]:
# for each rental_id, we merge in the same line, the values of the prevous rental id
df_merge = df.merge(df, how='inner', left_on='previous_rental_id', right_on='rental_id', suffixes=('', '_previous'))
df_merge

,rental_id,car_id,checkin_type,state,delay_checkout,previous_rental_id,delta_previous,rental_id_previous,car_id_previous,checkin_type_previous,state_previous,delay_checkout_previous,previous_rental_id_previous,delta_previous_previous
0,511639,370585,connect,ended,-15.0,563782.0,570.0,563782,370585,connect,ended,136.0,NaN,NaN
1,519491,312389,mobile,ended,58.0,545639.0,420.0,545639,312389,mobile,ended,140.0,NaN,NaN
2,521156,392479,mobile,ended,NaN,537298.0,0.0,537298,392479,mobile,ended,NaN,NaN,NaN
3,525044,349751,mobile,ended,NaN,510607.0,60.0,510607,349751,mobile,ended,-113.0,NaN,NaN
4,528808,181625,connect,ended,-76.0,557404.0,330.0,557404,181625,connect,ended,-352.0,554077.0,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1836,567741,294059,mobile,ended,111.0,567708.0,120.0,567708,294059,mobile,ended,283.0,NaN,NaN
1837,568241,396409,mobile,canceled,NaN,566136.0,570.0,566136,396409,mobile,ended,16.0,NaN,NaN
1838,568523,297973,mobile,ended,12.0,567121.0,240.0,567121,297973,mobile,ended,-275.0,NaN,NaN
1839,569717,377312,mobile,ended,230.0,545045.0,90.0,545045,377312,mobile,ended,22.0,563963.0,630.0


In [98]:
df_merge = df_merge[['rental_id','previous_rental_id','car_id','state','checkin_type','checkin_type_previous','delay_checkout_previous','delta_previous']]

In [99]:
df_merge = df_merge.dropna()

In [100]:
df_merge

,rental_id,previous_rental_id,car_id,state,checkin_type,checkin_type_previous,delay_checkout_previous,delta_previous
0,511639,563782.0,370585,ended,connect,connect,136.0,570.0
1,519491,545639.0,312389,ended,mobile,mobile,140.0,420.0
3,525044,510607.0,349751,ended,mobile,mobile,-113.0,60.0
4,528808,557404.0,181625,ended,connect,connect,-352.0,330.0
5,533670,556563.0,320824,ended,connect,connect,23.0,630.0
...,...,...,...,...,...,...,...,...
1835,573274,571227.0,298117,ended,connect,connect,-13.0,210.0
1836,567741,567708.0,294059,ended,mobile,mobile,283.0,120.0
1837,568241,566136.0,396409,canceled,mobile,mobile,16.0,570.0
1838,568523,567121.0,297973,ended,mobile,mobile,-275.0,240.0


In [101]:
px.histogram(df_merge, x='delay_checkout_previous', title='Distribution of delays')

In [102]:
px.histogram(df_merge, x='delta_previous', title='Distribution of delta', nbins=30)


In [103]:
from scipy import stats
line_count = df_merge.shape[0]
df_zscore = stats.zscore(df_merge.loc[:,'delay_checkout_previous'])
df_final = df_merge[np.abs(df_zscore) < 2]
print(f'{line_count-df_final.shape[0]} lines where dropped because of Outliers')

37 lines where dropped because of Outliers


In [104]:
px.histogram(df_final, x='delay_checkout_previous', title='Distribution of delays', nbins=200)

In [114]:
# we see that most of positive delays are between 0 and 170 min
max_delay = 59

In [115]:
# how much courses we could missed: the part of delta < max_delay. That's mean all if delta between to rental is < 170, we'll not got the rental because security is 170
no_course = df_final.query('delta_previous <= @max_delay').shape[0]
total_course = df_final.shape[0]
print('% of missed rental: ', (no_course / total_course * 100), 'and number of', no_course, 'courses missed')

% of missed rental:  22.22222222222222 and number of 376 courses missed


In [116]:
# how much courses we can prevent rental cancellation issue because of late checkout, comparing if we don't put any max_delay (0)
saved_course = df_final.query('(delay_checkout_previous <= @max_delay) & (delay_checkout_previous > 0)').shape[0]
print('% of saved courses: ', (saved_course / total_course * 100), ' and number of', saved_course, ' saved courses')

not_saved = df_final.query('(delay_checkout_previous > @max_delay)').shape[0]
print('% of not saved courses: ', (not_saved / total_course * 100), ' and number of', not_saved, ' not saved courses -> really bad user experience')

# todo : analyse the % of cancellation of if late checkout


% of saved courses:  21.513002364066196  and number of 364  saved courses
% of not saved courses:  29.550827423167846  and number of 500  not saved courses -> really bad user experience


In [108]:
# maximise the positive distance between no_course and saved_course
def get_best_delay_max(df_temp, filter = ''):
    distance_max = 0
    best_max_delay = 0
    no_rental_final = 0
    saved_rental_final = 0

    if filter != '':
        df_temp = df_temp.query(filter)
    total_rental = df_temp.shape[0]
    
    for max_delay_rental in range(1, 300):
        no_rental = df_temp.query('delta_previous <= @max_delay_rental').shape[0]
        saved_rental = df_temp.query('(delay_checkout_previous <= @max_delay_rental) & (delay_checkout_previous > 0)').shape[0]

        if (saved_rental - no_rental) > distance_max:
            distance_max = saved_rental - no_rental
            best_max_delay = max_delay_rental
            no_rental_final = no_rental
            saved_rental_final = saved_rental

    print('% of missed rental: ', (no_rental_final / total_rental * 100), 'and number of', no_rental_final, 'courses missed')
    print('% of saved courses: ', (saved_rental_final / total_rental * 100), ' and number of', saved_rental_final, ' saved courses')
    print('Distance between saved and missed: ', distance_max)

    return best_max_delay

In [109]:
# but saved_rental could be more to reduce even more the really bad customer experience to support. Even if it's increase the loss of revenue.
# It could be to analyse the lost of customers not using the app after this bad experience

In [110]:
get_best_delay_max(df_final, filter='checkin_type == "connect"')

% of missed rental:  22.750642673521853 and number of 177 courses missed
% of saved courses:  25.19280205655527  and number of 196  saved courses
Distance between saved and missed:  19


58

In [111]:
get_best_delay_max(df_final, filter='checkin_type == "mobile"')
# more useful for mobile

% of missed rental:  21.772428884026258 and number of 199 courses missed
% of saved courses:  36.542669584245075  and number of 334  saved courses
Distance between saved and missed:  135


59

In [112]:
# cancellation % if too much delay? NO, optionnal because : Customer service often reports users unsatisfied because they had to wait for the car to come back from the previous rental or users that even had to cancel their rental because the car wasn’t returned on time.
max_perc_ended = 0
best_delay_ended = 0
for i in range(30, 300):
    ended = df_final.query('(delay_checkout_previous > @i)').value_counts('state')['ended']
    canceled = df_final.query('(delay_checkout_previous > @i)').value_counts('state')['canceled']
    perc = (ended - canceled) / (ended + canceled)
    if perc > max_perc_ended:
        max_perc_ended = perc
        best_delay_ended = i

print(best_delay_ended)
print(1 - max_perc_ended)

42
0.2716627634660421


In [113]:
# cancellation % if too much delay?
df_final.query('(delay_checkout_previous > 50)').value_counts('state')

state
ended       324
canceled     52
dtype: int64